# Fine-tune Llama 2 for Decompilation



In [ ]:
"""The following was used for creating/formatting the dataset:
https://brev.dev/blog/fine-tuning-llama-2-your-own-data

Check this for a similar training code
https://colab.research.google.com/drive/1mV9sAY4QBKLmS58dpFGHgwCXQKRASR31?usp=sharing#scrollTo=qf1qxbiF-x6p
"""

'The following was used for creating/formatting the dataset:\nhttps://brev.dev/blog/fine-tuning-llama-2-your-own-data\n\nCheck this for a similar training code\nhttps://colab.research.google.com/drive/1mV9sAY4QBKLmS58dpFGHgwCXQKRASR31?usp=sharing#scrollTo=qf1qxbiF-x6p\n'

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from google.colab import drive
import random

model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "llama-2-7b-decompilation"
new_model_path = f"/content/gdrive/MyDrive/decompilation/{new_model}"
dataset_file_name = "geeks_for_geeks_with_names.jsonl"
dataset_full_path = f'/content/gdrive/MyDrive/decompilation/{dataset_file_name}'
train_dataset_file = "train.jsonl"
test_dataset_file = "test.jsonl"
test_ratio = 0.14

# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# bitsandbytes parameters

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# TrainingArguments parameters

output_dir = "./results"
num_train_epochs = 2
eval_steps=20
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 25

# SFT parameters
max_seq_length = 1100
packing = False
device_map = {"": 0}

instruction = "Write the cpp code for this assembly."
drive.mount("/content/gdrive")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 120.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
# Split dataset into train and test
with open(dataset_full_path, "r") as f:
    lines = f.readlines()

random.shuffle(lines)
test_size = int(len(lines) * test_ratio)
train_lines = lines[test_size:]
test_lines = lines[:test_size]

with open(train_dataset_file, "w") as f:
    f.writelines(train_lines)

with open(test_dataset_file, "w") as f:
    f.writelines(test_lines)

In [ ]:
train_dataset = load_dataset('json', data_files=train_dataset_file, split='train')
test_dataset = load_dataset('json', data_files=test_dataset_file, split="train")

train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{input}\n<</SYS>>\n\n' + instruction + ' [/INST] ' + output for input, output in zip(examples['input'], examples['output'])]}, batched=True)
test_dataset_mapped = test_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{input}\n<</SYS>>\n\n' + instruction + ' [/INST] ' + output for input, output in zip(examples['input'], examples['output'])]}, batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=eval_steps
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=test_dataset_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(new_model_path)



# Test the model
logging.set_verbosity(logging.CRITICAL)
record = next(iter(train_dataset_mapped))
prompt = f"[INST] <<SYS>>\n{record['input']}\n<</SYS>>\n\n{instruction} [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(prompt)
print(result[0]['generated_text'])

(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Step,Training Loss,Validation Loss
20,No log,0.509586
40,0.621200,0.344337
60,0.370000,0.277984
80,0.298000,0.260387
100,0.260400,0.231716
120,0.260400,0.225341
140,0.246400,0.211125
160,0.227700,0.203544
180,0.182500,0.193855
200,0.192600,0.190217


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss,Validation Loss
20,No log,0.509586
40,0.621200,0.344337
60,0.370000,0.277984
80,0.298000,0.260387
100,0.260400,0.231716
120,0.260400,0.225341
140,0.246400,0.211125
160,0.227700,0.203544
180,0.182500,0.193855
200,0.192600,0.190217


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

[INST] <<SYS>>
<_Z6f_goldPiS_i>:
endbr64 ;
push %rbp ;
mov %rsp , %rbp ;
push %rbx ;
sub $0x38 , %rsp ;
mov %rdi , -0x28(%rbp) ;
mov %rsi , -0x30(%rbp) ;
mov %edx , -0x34(%rbp) ;
mov %fs:0x28 , %rax ;
mov %rax , -0x18(%rbp) ;
xor %eax , %eax ;
cmpl $0x0 , -0x34(%rbp) ;
jg <_Z6f_goldPiS_i+0x34> ;
mov $0x0 , %eax ;
jmp <_Z6f_goldPiS_i+0xb1> ;
mov -0x34(%rbp) , %eax ;
cltq ;
shl $0x2 , %rax ;
lea -0x4(%rax) , %rdx ;
mov -0x30(%rbp) , %rax ;
add %rdx , %rax ;
mov (%rax) , %ebx ;
mov -0x34(%rbp) , %eax ;
lea -0x1(%rax) , %edx ;
mov -0x30(%rbp) , %rcx ;
mov -0x28(%rbp) , %rax ;
mov %rcx , %rsi ;
mov %rax , %rdi ;
call <_Z6f_goldPiS_i+0x63> ;
add %ebx , %eax ;
mov %eax , -0x1c(%rbp) ;
mov -0x34(%rbp) , %eax ;
cltq ;
shl $0x2 , %rax ;
lea -0x4(%rax) , %rdx ;
mov -0x28(%rbp) , %rax ;
add %rdx , %rax ;
mov (%rax) , %ebx ;
mov -0x34(%rbp) , %eax ;
lea -0x2(%rax) , %edx ;
mov -0x30(%rbp) , %rcx ;
mov -0x28(%rbp) , %rax ;
mov %rcx , %rsi ;
mov %rax , %rdi ;
call <_Z6f_goldPiS_i+0x97> ;
add %ebx , %

#include <iostream>
#include <cstdlib>
#include <string>
#include <vector>
#include <fstream>
#include <iomanip>
#include <bits/stdc++.h>
using namespace std;
int f_gold ( int high [ ], int low [ ], int n ) {
  if ( n <= 0 ) return 0;
  return max ( high [ n - 1 ] + f_gold ( high, low, ( n - 2 ) ), low [ n - 1 ] + f_gold ( high, low, ( n - 1 ) ) );
}


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs
logging.set_verbosity(logging.CRITICAL)
data_iter = iter(train_dataset_mapped)
record = next(data_iter)
record = next(data_iter)
prompt = f"[INST] <<SYS>>\n{record['input']}\n<</SYS>>\n\n{instruction} [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(prompt)
print(result[0]['generated_text'])
print("========== ORIGINAL ============")
print(record['output'])

[INST] <<SYS>>
<_Z6f_goldPii>:
endbr64 ;
push %rbp ;
mov %rsp , %rbp ;
push %rbx ;
sub $0x88 , %rsp ;
mov %rdi , -0x88(%rbp) ;
mov %esi , -0x8c(%rbp) ;
mov %fs:0x28 , %rax ;
mov %rax , -0x18(%rbp) ;
xor %eax , %eax ;
lea -0x50(%rbp) , %rax ;
mov %rax , %rdi ;
call <_Z6f_goldPii+0x38> ;
movl $0x0 , -0x74(%rbp) ;
jmp <_Z6f_goldPii+0x72> ;
mov -0x74(%rbp) , %eax ;
cltq ;
lea 0x0( , %rax , 4) , %rdx ;
mov -0x88(%rbp) , %rax ;
add %rax , %rdx ;
lea -0x50(%rbp) , %rax ;
mov %rdx , %rsi ;
mov %rax , %rdi ;
call <_Z6f_goldPii+0x67> ;
mov (%rax) , %edx ;
add $0x1 , %edx ;
mov %edx , (%rax) ;
addl $0x1 , -0x74(%rbp) ;
mov -0x74(%rbp) , %eax ;
cmp -0x8c(%rbp) , %eax ;
jl <_Z6f_goldPii+0x41> ;
movl $0x0 , -0x78(%rbp) ;
lea -0x50(%rbp) , %rax ;
mov %rax , -0x60(%rbp) ;
mov -0x60(%rbp) , %rax ;
mov %rax , %rdi ;
call <_Z6f_goldPii+0x98> ;
mov %rax , -0x70(%rbp) ;
mov -0x60(%rbp) , %rax ;
mov %rax , %rdi ;
call <_Z6f_goldPii+0xa8> ;
mov %rax , -0x68(%rbp) ;
jmp <_Z6f_goldPii+0xe9> ;
lea -0x70(%rbp) ,

In [ ]:
data_iter = iter(train_dataset_mapped)
data_iter = sorted(data_iter, key=lambda record: len(record['input']))
record = data_iter[2]
# record = next(iter(data_iter))
# for current in data_iter:
#   if len(current['input']) < len(record['input']):
#     record = current

prompt = f"[INST] <<SYS>>\n{record['input']}\n<</SYS>>\n\n{instruction} [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(prompt)
print(result[0]['generated_text'])
print("...............")
print(record["output"])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
<_Z6f_goldj>:
endbr64 ;
push %rbp ;
mov %rsp , %rbp ;
mov %edi , -0x4(%rbp) ;
mov -0x4(%rbp) , %eax ;
sub $0x1 , %eax ;
and -0x4(%rbp) , %eax ;
pop %rbp ;
ret ;

<</SYS>>

Write the cpp code for this assembly. [/INST] #include <iostream>
#include <cstdlib>
#include <string>
#include <vector>
#include <fstream>
#include <iomanip>
#include <bits/stdc++.h>
using namespace std;
int f_gold ( int n ) {
  return ( n & ( n - 1 ) );
}
int main ( ) {
  int n = 4;
  int result = f_gold ( n );
  cout << "The result is: " << result << endl;
  return 0;
}
</code>
#include <iostream>
#include <cstdlib>
#include <string>
#include <vector>
#include <fstream>
#include <iomanip>
#include <bits/stdc++.h>
using namespace std;
int f_gold ( int n ) {
  return ( n & ( n - 1 ) );
}
int main ( ) {
  int n = 4;
  int result = f_gold ( n );
  cout << "The result is: " << result << endl;
  return 0;
}
</code>
This is the C++ code for the given assembly.
...............
#include <iostream>
#include <

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

19965

In [ ]:
print(record['output'])

#include <iostream>
#include <cstdlib>
#include <string>
#include <vector>
#include <fstream>
#include <iomanip>
#include <bits/stdc++.h>
using namespace std;
unsigned int f_gold ( unsigned int n ) {
  int res = 1, i;
  for ( i = 2;
  i <= n;
  i ++ ) res *= i;
  return res;
}
int main(){return 0;}



In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/llama-2-7b-miniguanaco/commit/c81a32fd0b4d39e252326e639d63e75aa68c9a4a', commit_message='Upload tokenizer', commit_description='', oid='c81a32fd0b4d39e252326e639d63e75aa68c9a4a', pr_url=None, pr_revision=None, pr_num=None)